In [1]:
import json

file_path = "links.txt"

base_url = "https://www.paulgraham.com/"

with open(file_path) as f:
    links = json.load(f)

full_links = [base_url + link for link in links]

print(full_links)

['https://www.paulgraham.com/greatwork.html', 'https://www.paulgraham.com/kids.html', 'https://www.paulgraham.com/selfindulgence.html', 'https://www.paulgraham.com/field.html', 'https://www.paulgraham.com/goodwriting.html', 'https://www.paulgraham.com/do.html', 'https://www.paulgraham.com/woke.html', 'https://www.paulgraham.com/writes.html', 'https://www.paulgraham.com/when.html', 'https://www.paulgraham.com/foundermode.html', 'https://www.paulgraham.com/persistence.html', 'https://www.paulgraham.com/reddits.html', 'https://www.paulgraham.com/google.html', 'https://www.paulgraham.com/best.html', 'https://www.paulgraham.com/superlinear.html', 'https://www.paulgraham.com/getideas.html', 'https://www.paulgraham.com/read.html', 'https://www.paulgraham.com/want.html', 'https://www.paulgraham.com/alien.html', 'https://www.paulgraham.com/users.html', 'https://www.paulgraham.com/heresy.html', 'https://www.paulgraham.com/words.html', 'https://www.paulgraham.com/goodtaste.html', 'https://www.pau

In [2]:
from dotenv import load_dotenv

load_dotenv()

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

hf = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
)

/home/bytehead/projects/temp/rag/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="PG_essays",
    embedding_function=hf,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [8]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from datetime import datetime

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=full_links,
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(["title", "body"])),
)
try:
    docs = loader.load()
    for doc in docs:
        doc.metadata["chunk_source"] = "web scraping"
        doc.metadata["processing_date"] = str(datetime.now())
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    all_splits = text_splitter.split_documents(docs)
    print(f"Loaded {len(all_splits)} chunks from {len(docs)} documents.")
except Exception as e:
    print(f"An error occurred while loading or processing documents: {e}")

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
prompt = hub.pull("rlm/rag-prompt")

Loaded 4161 chunks from 228 documents.


In [9]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [10]:
response = graph.invoke({"question": "Why do startups fail?"})
print(response["answer"])

The primary reason startups fail is not making something users want. This fundamental mistake can stem from various factors, including having a single founder. Additionally, slow growth or excessive spending can lead to startups running out of funding.


In [11]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

In [21]:
from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=5)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [22]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks from Paul Graham's essays and are called AskPG. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [23]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid4

memory = MemorySaver()

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile(checkpointer=memory)

config = {"configurable": {"thread_id": str(uuid4())}}

Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.


ValueError: Node `query_or_respond` already present.

In [24]:
input_message = "How to make sure my startup wont fail?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

How to make sure my startup wont fail?
================================== Ai Message ==================================

To increase your startup's chances of success, consider these key points:

1.  **Iterate on your ideas:** Don't treat your initial idea as set in stone. Many valuable ideas will emerge during the implementation process.
2.  **Understand your users deeply:** Focus on truly understanding what your users lack and how you can improve their lives. This understanding is crucial for creating something they genuinely need.
3.  **Aim for love, not just acceptance:** It's better to have a small group of users who absolutely love your product than a large group who are indifferent. This strong initial engagement can drive further growth.
4.  **Do things that don't scale:** Startups often need a "manual push" to get going. This might involve actively recruiting users one-by-one rather than waiting 